In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import time
import tdmclient
import tdmclient.notebook
import threading

import camera as cam
import vision_utils as utils
import Global_tomerge as global_nav
import local_nav as local_nav

In [2]:
global robot,goal, object_corners
object_corners =[]
robot=()
goal=()





**Navigation Thread**

In [3]:
await tdmclient.notebook.start()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
@tdmclient.notebook.sync_var
def thymio_speed(speed_left,speed_right):
    global speed_left_global
    global speed_right_global
    speed_right_global = speed_right
    speed_left_global = speed_left

@tdmclient.notebook.sync_var
def thymio_state(has_global_path):
    global state_global
    thymio_state = "Local"
    if state_global == 1:
        thymio_state = "Avoidance"
        has_global_path = False
    return thymio_state, has_global_path

def thymio_turn(angle_diff):
    print("Turning ", angle_diff, " radians")
    if abs(angle_diff) > 10:
        if angle_diff > 0:
            thymio_speed(-50,50)
            time.sleep(0.05*angle_diff)
            thymio_speed(0,0)
            time.sleep(0.1)
        else:
            thymio_speed(50,-50)
            time.sleep(0.05*abs(angle_diff))
            thymio_speed(0,0)
            time.sleep(0.1)
        return angle_diff*math.pi/180 # return in radians
    return 0

@tdmclient.notebook.sync_var
def print_thymio_speed():
    global motor_left_speed, motor_right_speed
    mls = motor_left_speed
    mrs = motor_right_speed
    print("Left motor speed: ", mls)
    print("Right motor speed: ", mrs)

In [ ]:
thymio_speed(0,0)

In [ ]:
print_thymio_speed()

In [ ]:
%%run_python
speed_left_global = 0
speed_right_global = 0
obstSpeedGain = [6, 4, -2, -6, -8]
state_global = 0
DELTA = 1
timer_period[0] = 10  # 10ms sampling time

@onevent 
def timer0():
    global prox_ground_delta, prox_horizontal, speed0, speedGain,obstSpeedGain, motor_left_target, motor_right_target
    global speed_left_global, speed_right_global, state_global, motor_left_speed, motor_right_speed
    spLeft = speed_left_global
    spRight = speed_right_global
    
    # adjustment for obstacles ("gradient" due to obstacles)
    for i in range(5):
        spLeft += prox_horizontal[i] * obstSpeedGain[i] // 100
        spRight += prox_horizontal[i] * obstSpeedGain[4 - i] // 100
    # motor control
    motor_left_target = spLeft
    motor_right_target = spRight

    if spLeft - spRight < DELTA and spRight - spLeft < DELTA:
        state_global = 0
    else:
        state_global = 1



In [ ]:
def nav_thread():
    global robot,goal, object_corners, angle_robot,path
    state = 'Local'
    has_gobal_path = True
    robot = None
    goal = None
    angle_robot = 0
    robot_size = 23
    run = True
    counter = 0
    path = []
    print('nav_thread')
    while run:
        state,has_gobal_path = thymio_state(has_gobal_path)
        if state == "Local":
            if not has_gobal_path:
                # print('global')
                if robot is not None and goal != []:
                    print('robot')
                    RandG = global_nav.create_RandG_dict(robot, goal)
                    object_corners = global_nav.create_dictionnary(object_corners)
                    print("R+G", RandG)
                    print("object", object_corners)
                    expended_corners = global_nav.grow_obstacles(object_corners, robot_size)
                    points_name2coord = global_nav.name2coord(expended_corners, RandG)
                    adjacent_list = global_nav.generate_adjacency_list(expended_corners, RandG)
                    distances = global_nav.calculate_distances(adjacent_list, points_name2coord)
                    shortest_path = global_nav.find_path(adjacent_list, points_name2coord)
                    for name in shortest_path:
                        path.append(points_name2coord[name])
                    has_gobal_path = True
                    counter = 0
                    if path != []:
                        x_pos = path[0][0]
                        y_pos = path[0][1]
                        theta = angle_robot
                        initial_state = np.array([x_pos, y_pos, 0.0, 0.0, theta, 0])
                        initial_covariance = np.diag([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])  # Initial covariance matrix
                        process_noise = np.diag([0.03, 0.03, 0.01, 0.01, 0.02, 0.01])  # Process noise covariance matrix
                        measurement_noise = np.array([0.1, 0.1])  # Measurement noise variance

                        # Initialize Kalman filter
                        local_state = initial_state
                        covariance = initial_covariance   
                        time_prev = time.time()
                        new_left_speed = 0
                        new_right_speed = 0
                        cor = 2 #correction factor for the target coordinates (1cm)
                        
        # for local state
        if path != []:
            print(path)
            
            coordinate = path[counter+1]

            if local_nav.check_target(local_state, coordinate, cor) == True:
                cam_working = robot is not None 
                if cam_working and angle_robot is not None:
                    theta = angle_robot
                else:
                    theta = local_state[4]

                left_speed = new_left_speed
                right_speed = new_right_speed
                input_val = np.array([local_nav.convert_speed(left_speed), local_nav.convert_speed(right_speed)])

                Ts = time.time() - time_prev
                time_prev = time.time()

                local_state, covariance = local_nav.predict(local_state, input_val, covariance, Ts, theta, process_noise)              
                local_state, covariance = local_nav.measure(local_state, covariance, cam_working, measurement_noise, np.array(robot))               
                
                if (local_state[0], local_state[1]) == path[0]:
                    local_state, covariance = local_nav.predict(local_state, input_val, covariance, Ts, theta, process_noise)              
                    local_state, covariance = local_nav.measure(local_state, covariance, cam_working, measurement_noise, np.array(robot))                

                print("local_state:", local_state)

                if local_nav.check_target(local_state, coordinate, cor) == True: # when the target is reached, it doesn't need to turn anymore since that would lead to the robot spinning
                    print('goal not reached')
                    print(theta)
                    #angle_diff = local_nav.turn_to_target((local_state[0],local_state[1]), coordinate, theta)
                    #thymio_turn(angle_diff)
                    #theta = theta + angle_diff
                    
                    
                    new_left_speed, new_right_speed = local_nav.go_to_target((local_state[0],local_state[1]), coordinate, theta)
                    thymio_speed(new_left_speed, new_right_speed)

                else:
                    print("goal reached")
                    thymio_speed(0, 0)

            else:
                counter += 1
                if counter == len(path)-1:
                    run = False
                    thymio_speed(0,0)
                    break


In [ ]:
nav_thread = threading.Thread(target=nav_thread)

***Camera Thread***

In [2]:
import traceback
def camera_thread():
    global robot, goal, object_corners, angle_robot, cov_noise ,height, path       

    try:
        camera_index = 0
        # If you are to use the cam object, make sure it is defined and initialized
        # data = cam.capture_data(camera_index)
        # cov_noise = cam.calculate_covariance_matrix(data)
        cap = cv2.VideoCapture(camera_index)
        
        if not cap.isOpened():
            print("Error opening video stream or file")
            return

        mode = 'all'
        color_type = 'RGB'

        # Define thresholds initially
        threshold  = [cam.COLOR_THRESHOLD,cam.SATURATION_THRESHOLD,cam.BRIGHTNESS_THRESHOLD]

        cv2.namedWindow("Camera Vision", cv2.WINDOW_NORMAL)

        # Create Trackbars
        cv2.createTrackbar("Color Threshold", "Camera Vision", threshold[0], 200,
                        lambda x: utils.on_trackbar(x, threshold, 0))
        cv2.createTrackbar("Saturation Threshold", "Camera Vision", threshold[1], 200,
                        lambda x: utils.on_trackbar(x, threshold, 1))
        cv2.createTrackbar("Brightness Threshold", "Camera Vision", threshold[2], 200,
                        lambda x: utils.on_trackbar(x, threshold, 2))
        height = 100
        width =75
        origin = (0,0)
        path = []
        M = None

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cam.zoom_frame(frame,zoom_factor=1)
            #print("Camera Path: ", path)
            # Get updated threshold values from the trackbars
            threshold_color = cv2.getTrackbarPos("Color Threshold", "Camera Vision")
            threshold_saturation = cv2.getTrackbarPos("Saturation Threshold", "Camera Vision")
            threshold_brightness = cv2.getTrackbarPos("Brightness Threshold", "Camera Vision")

            # Camera vision
            frame, robot, angle_robot, goal, object_corners, map_corners,width, height,origin,M = cam.detection(
                frame,
                mode,
                color_type,
                threshold[0],  # Color threshold
                threshold[1],  # Saturation threshold
                threshold[2],  # Brightness threshold
                width,
                height,
                origin,
                M
            )
            cv2.imshow("Camera Vision", frame)
            print("Robot: ", robot)
            print("Angle: ", angle_robot)

            key = cv2.waitKey(25) & 0xFF
            if key == 27:  # ESC key to exit
                break
            elif key == ord('1'):  # '1' key to switch to blue mode
                mode = 'blue'
            elif key == ord('2'):  # '2' key to switch to green mode
                mode = 'green'
            elif key == ord('3'):  # '3' key to switch to black mode
                mode = 'black'
            elif key == ord('4'):  # '4' key to switch to black mode
                mode = 'red'
            elif key == ord('5'):  # '5' key to switch to RGB mode
                color_type = 'RGB'
            elif key == ord('6'):  # '6' key to switch to BGR mode
                color_type = 'BGR'
            elif key == ord('7'):
                mode = 'all'

            time.sleep(0.01)

    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("Done displaying the video")


In [3]:
camera_thread()

Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
Robot circle None
Robot:  None
Angle:  None
304.5606814689324 360.1096557405724 xy
(457.0, 305.00000000000006)
Robot circle (-55.10965574057235, 152.4393185310676)
Robot:  (66.64347515731562, 88.55155469030467)
Angle:  0.009729990130215091
304.29661321172523 362.5837355034842 xy
(457.99999999999994, 306.00000000000006)
Robot circle (-56.58373550348415, 153.7033867882747)
Robot:  (66.44030856151207, 88.86856262340297)
Angle:  0.054654018147103835
304.19303679933734 362

In [ ]:
camera_thread = threading.Thread(target=camera_thread)

In [ ]:
camera_thread.start()

In [ ]:
nav_thread.start()

In [ ]:
camera_thread.join()

In [ ]:
nav_thread.join()